In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from pathlib import Path

import datasets
import h5py
import matplotlib.pyplot as plt
import transformers
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from src.models.frame_level import FrameLevelLexicalAccess, \
    LexicalAccessConfig, LexicalAccessDataCollator
from src.models.transformer import drop_wav2vec_layers

In [5]:
regressor_target_size = 32

In [43]:
def make_model_init(
        model_name_or_path,
        config, 
        word_vocabulary, word_representations,
        device="cpu"):
    def model_init(trial):
        encoder = transformers.Wav2Vec2Model.from_pretrained(
            model_name_or_path, config=config.encoder_config).to(device)
        model = FrameLevelLexicalAccess(
            config, word_vocabulary, word_representations,
            encoder=encoder).to(device)

        model.freeze_feature_extractor()

        if hasattr(config, "drop_layers"):
            model.encoder = drop_wav2vec_layers(model.encoder, config.drop_layers)

        if getattr(config, "reinit_feature_extractor_weights", False):
            model.encoder.feature_extractor.apply(lambda x: model.encoder._init_weights(x))
        if getattr(config, "reinit_encoder_weights", False):
            model.encoder.encoder.apply(lambda x: model.encoder._init_weights(x))

        # Freeze all model weights.
        for param in model.encoder.parameters():
            param.requires_grad = False
        
        return model
    return model_init

In [7]:
tokenizer = transformers.Wav2Vec2Tokenizer.from_pretrained(
    "charsiu/tokenizer_en_cmu")
feature_extractor = transformers.Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000)
processor = transformers.Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/userdata/jgauthier/transformers/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:792: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
dataset = datasets.load_from_disk("./data/timit_phoneme/")
dataset_split = dataset["train"].train_test_split(test_size=0.1, shuffle=True)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

## Prepare semantic representations

In [40]:
all_words = set()
def update_all_words(item):
    all_words.update(set(item["word_detail"]["utterance"]))
    return None
dataset.map(update_all_words)

all_words = sorted(all_words)
all_words[:10]

Map:   0%|          | 0/4620 [00:00<?, ? examples/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

["'em",
 'a',
 'abbreviate',
 'abdomen',
 'abides',
 'ability',
 'able',
 'ably',
 'abolish',
 'aborigine']

In [41]:
word_representations = np.random.randint(
    0, 2, (len(all_words), regressor_target_size))
word_representations = torch.tensor(word_representations, dtype=torch.float32)

In [42]:
word_representations.shape

torch.Size([6102, 32])

## Run

In [21]:
encoder_config = transformers.AutoConfig.from_pretrained(
    "facebook/wav2vec2-base")
model_config = LexicalAccessConfig(
    encoder_config=encoder_config,
    classifier_num_labels=tokenizer.vocab_size,
    regressor_target_size=regressor_target_size)

/userdata/jgauthier/transformers/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [22]:
tokenizer = transformers.Wav2Vec2Tokenizer.from_pretrained("charsiu/tokenizer_en_cmu")
model = transformers.Wav2Vec2ForCTC.from_pretrained("charsiu/en_w2v2_ctc_libris_and_cv")
feature_extractor = transformers.Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = transformers.Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/userdata/jgauthier/transformers/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:792: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
model_init = make_model_init(
    "charsiu/en_w2v2_ctc_libris_and_cv", model_config,
    all_words, word_representations)

In [48]:
collator = LexicalAccessDataCollator(
    processor=processor,
    model=model_init(None),
    padding=True,
    num_labels=tokenizer.vocab_size,
    regression_target_size=regressor_target_size)

In [52]:
next(iter(collator(train_dataset.select(range(2)))))

'input_values'